In [31]:
# Dependencies
import pandas as pd
import requests
import time
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import datetime as dt

In [32]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path)

In [48]:
url_nasa = "https://mars.nasa.gov/news"
url_jpl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_weather = "https://twitter.com/marswxreport?lang=en"
url_facts = "https://space-facts.com/mars"
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [49]:
browser.visit(url_nasa)

In [50]:
html = browser.html
soup = bs(html, "html.parser")

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text.

In [54]:
# NASA MARS top news story - title
news_title = soup.find('div', class_="content_title").text.strip()
news_title

'The MarCO Mission Comes to an End'

In [53]:
# NASA MARS top news story - paragraph
news_p = soup.find('div', class_="article_teaser_body").text.strip()
news_p

'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'

In [36]:
response_nasa=requests.get(url_nasa)
r=response_nasa.json()
response_jpl=requests.get(url_jpl)
response_weather=requests.get(url_weather)
response_facts=requests.get(url_facts)
response_hemispheres=requests.get(url_hemispheres)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
soup_nasa=bs(response_nasa.text, "html.parser")
soup_jpl=bs(response_jpl.text, "html.parser")
soup_weather=bs(response_weather.text, "html.parser")
soup_facts=bs(response_facts.text, "html.parser")
soup_hemispheres=bs(response_hemispheres.text, "html.parser")

In [ ]:
# The article header
news_title = soup.find('div', class_="content_title").text
print(news_title)

In [ ]:
print(soup_nasa.prettify())

In [ ]:
news_title=r['items'][0]['title']
print(news_title)
news_p=r['items'][0]['description']
print(news_p)

In [ ]:
result_img=soup_jpl.find('img', class_='thumb')
print(result_img)

In [ ]:
featured_image_url='https://www.jpl.nasa.gov'+result_img['src']
print(featured_image_url)

In [ ]:
mars_weather=soup_weather.find_all('p')[4].text
print(mars_weather)
mars_weather=mars_weather.replace('\n', ' ')
mars_weather

In [ ]:
tables = pd.read_html(url_facts)
tables

In [ ]:
print(tables[0])

In [ ]:
df=tables[0]
df.columns=['description','value']
df.head()

In [ ]:
html_table = df.to_html()
html_table

In [ ]:
#to save table directly to a file
df.to_html('table.html')

In [ ]:
results=soup_hemispheres.find_all('a', class_='itemLink product-item')
results

In [ ]:
hemisphere_image_urls=[]
for result in results:
    title=result.h3.text
    
    url=result.img['src']
    
    full_url="https://astrogeology.usgs.gov"+url
    hemisphere_image_urls.append({"title":title, "img_url": full_url})
print(hemisphere_image_urls)